In [1]:
import pandas as pd
import numpy as np

zcd_ABE_HEK293T = pd.read_csv('ABE_HEK293T_3_10.csv')
print(len(zcd_ABE_HEK293T))
df_ = zcd_ABE_HEK293T.copy()
'''
ATTENTION: make sure datas in one group are splited to the same part

    For example:
    
seq1 outcome1
seq1 outcome2
seq1 outcome3
seq2 outcome4
seq3 outcome5
seq3 outcome6

don't leave 'outcome 1,2' are like, one in test dataset and another in training dataset
'''

### get the 10% groups as test datasets ###

grp = df_.groupby('Reference')

num_grps = len(grp)
num_to_select = max(1, int(0.1 * num_grps))

selected_groups = np.random.choice(list(grp.groups.keys()), num_to_select, replace=False)

test_zcd_ABE = df_[df_['Reference'].isin(selected_groups)]
test_zcd_ABE = test_zcd_ABE.rename(columns = {'Reference':'Input','Outcome': 'Output', 'Outcome_Frequency':'True_Proportion'})
test_zcd_ABE = test_zcd_ABE[['Input','Output', 'True_Proportion']]

test_zcd_ABE = test_zcd_ABE.reset_index(drop = True)

test_zcd_ABE
## zcd_ABE: as the test datasets
## output should be around 16,000 sequences

164129


,Input,Output,True_Proportion
0,AAAACGGCAAGAAGAACAAG,AAAACGGCAGGAAGAACAAG,0.000000
1,AAAACGGCAAGAAGAACAAG,AAAACGGCGAGAAGAACAAG,0.000000
2,AAAACGGCAAGAAGAACAAG,AAAACGGCGGGAAGAACAAG,0.000000
3,AAAACGGCAAGAAGAACAAG,AAAGCGGCAAGAAGAACAAG,0.006757
4,AAAACGGCAAGAAGAACAAG,AAAGCGGCAGGAAGAACAAG,0.000000
...,...,...,...
16306,GTTTGTCAAGCTGTCCCGCG,GTTTGTCGAGCTGTCCCGCG,0.033520
16307,GTTTGTCAAGCTGTCCCGCG,GTTTGTCGGGCTGTCCCGCG,0.000000
16308,GTTTGTGAGATGTCTGAAGA,GTTTGTGAGGTGTCTGAAGA,0.012500
16309,GTTTGTGAGATGTCTGAAGA,GTTTGTGGGATGTCTGAAGA,0.012500


In [2]:
zcd_ABE_lib = zcd_ABE.drop_duplicates(subset = 'Input')
zcd_ABE_lib = zcd_ABE_lib.iloc[:,:1]
zcd_ABE_lib = zcd_ABE_lib.reset_index(drop = True)

### processed lib ###
### get lib ###

zcd_ABE_lib


### zcd_ABE_lib: as the lib with 50bp length ###
### 3289 rows × 1 columns ###

,Input
0,AAAAAGGCCTGCCCCATGTG
1,AAAACAGACGAAGGACCAGT
2,AAAAGAGACTCCAGGCACTA
3,AAAAGGAGACCAGTATATTG
4,AAAAGGCTCAACATTTACAC
...,...
3284,GTTTGCCAGCCAGCATCACA
3285,GTTTGCCAGGAAGCCTGTCG
3286,GTTTGGACTTGCACTCGAAG
3287,GTTTGGTCAATTGCAGGACA


In [8]:
# with open('OT_Lib_Final.txt','r') as f:
#     for i in range(10):
#         Lib = f.readline()
#         print(Lib.strip())

In [35]:
### read the lib txt file and convert to csv file ###
### need to be stored in repository ###


# Lib = pd.read_csv('OT_Lib_Final.txt', sep = '\t')
# print(f'lib column names:{Lib.columns}')


### recover to 50bp sequence ###

### get the Lib: Lib.columns = 'Seq' ###
### total 81,330 rows ###

df = pd.read_csv('all_lib.csv', index_col = 0)

print(len(df))

Lib = df[['Seq']].copy()
Lib['Seq'] = Lib['Seq'].str.upper()
# Lib['Seq'] = Lib['Seq'].str[110:160]
# Lib.to_csv('Lib_with_only_50bp_seq.csv')

81330


In [3]:
#### recover to 50bp seq ###

# if Lib['Seq'].str[20:40] == zcd_ABE['Input'], then substitute zcd_ABE['Input'] with Lib['Seq'].str[20:40]
# also get the matched rows that zcd_ABE['Output'] == Lib['seq'].str[0:20] + zcd_ABE['Output'] + Lib['seq'].str[40:50]

zcd_ABE_ = test_zcd_ABE[:3]
zcd_ABE_

,Input,Output,True_Proportion
0,AAAACGGCAAGAAGAACAAG,AAAACGGCAGGAAGAACAAG,0.0
1,AAAACGGCAAGAAGAACAAG,AAAACGGCGAGAAGAACAAG,0.0
2,AAAACGGCAAGAAGAACAAG,AAAACGGCGGGAAGAACAAG,0.0


In [33]:
lib

,gRNA_Name,gRNA_seq,gRNA_PAM,target_seq,Seq,target_pos,prefix,suffix,target_gRNA_PAM,count
0,hsa-mir-3129--81249,AAAAAAAAAAAGTCGTACTT,GAAAAAAAAAAAGTCGTACTTGGgt,ATGGAAAAAAAAAAAGTCGTACTTGGG,gcgagaaaagccttgtttAAAAAAAAAAAGTCGTACTTgttttaga...,130,gccaccATGG,tgagcaaggg,AAAAAAAAAAAGTCGTACTTGGG,1
1,hsa-mir-4329--79147,AAAAAAAAACTCCAAAACCC,GAAAAAAAAACTCCAAAACCCTGgt,ATGGAAAAAAAAACTCCAAAACCCTGG,gcgagaaaagccttgtttAAAAAAAAACTCCAAAACCCgttttaga...,130,gccaccATGG,tgagcaaggg,AAAAAAAAACTCCAAAACCCTGG,1
2,hsa-mir-1303--78858,AAAAAAAAATTGTAGAGTTG,GAAAAAAAAATTGTAGAGTTGAGgt,ATGGAAAAAAAAATTGTAGAGTTGAGG,gcgagaaaagccttgtttAAAAAAAAATTGTAGAGTTGgttttaga...,130,gccaccATGG,tgagcaaggg,AAAAAAAAATTGTAGAGTTGAGG,1
3,SLC25A24--36887,AAAAAAAATCCGGACAATGG,GAAAAAAAATCCGGACAATGGTGGG,ATGGAAAAAAAATCCGGACAATGGTGG,gcgagaaaagccttgtttAAAAAAAATCCGGACAATGGGttttaga...,130,gccaccATGG,Ggtgagcaag,AAAAAAAATCCGGACAATGGTGG,1
4,ZNF43--20560,AAAAAAAGACCATAAAAGTG,GAAAAAAAGACCATAAAAGTGTGgt,ATGGAAAAAAAGACCATAAAAGTGTGG,gcgagaaaagccttgtttAAAAAAAGACCATAAAAGTGGttttaga...,130,gccaccATGG,tgagcaaggg,AAAAAAAGACCATAAAAGTGTGG,1
...,...,...,...,...,...,...,...,...,...,...
81325,RIPK3--29898,GTTTGTTAACGTAAACCGGA,GGTTTGTTAACGTAAACCGGAAGGg,ATGGGGTTTGTTAACGTAAACCGGAAG,gcgagaaaagccttgtttGTTTGTTAACGTAAACCGGAGttttaga...,131,ccaccATGGG,gtgagcaagg,GTTTGTTAACGTAAACCGGAAGG,1
81326,BEND6--65735,GTTTGTTGCTCTCCAGAGGG,GGTTTGTTGCTCTCCAGAGGGTGgt,ATGGGTTTGTTGCTCTCCAGAGGGTGG,gcgagaaaagccttgtttGTTTGTTGCTCTCCAGAGGGGttttaga...,130,gccaccATGG,tgagcaaggg,GTTTGTTGCTCTCCAGAGGGTGG,1
81327,hsa-mir-8068--78791,GTTTGTTGTAAGGATCGTTG,GGTTTGTTGTAAGGATCGTTGTGgt,ATGGTTTGTTGTAAGGATCGTTGTGGT,gcgagaaaagccttgtttGTTTGTTGTAAGGATCGTTGgttttaga...,129,tgccaccATG,tgagcaaggg,GTTTGTTGTAAGGATCGTTGTGG,1
81328,OS9--29649,GTTTGTTGTTGGCATGATAG,GGTTTGTTGTTGGCATGATAGGGGg,ATGGTTTGTTGTTGGCATGATAGGGGG,gcgagaaaagccttgtttGTTTGTTGTTGGCATGATAGGttttaga...,129,tgccaccATG,gtgagcaagg,GTTTGTTGTTGGCATGATAGGGG,1


In [32]:
print(lib[lib['count'] == 0])

Empty DataFrame
Columns: [gRNA_Name, gRNA_seq, gRNA_PAM, target_seq, Seq, target_pos, prefix, suffix, target_gRNA_PAM, count]
Index: []


In [39]:
print(Lib.iloc[0, 0])
print(Lib.iloc[0, 0][0:18])

GCGAGAAAAGCCTTGTTTAAAAAAAAAAAGTCGTACTTGTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTATCAACTTGAAAAAGTGGCACCGAGTCGGTGCTTTTTTGCCACCATGGAAAAAAAAAAAGTCGTACTTGGGTGAGCAAGGGCGAGG
GCGAGAAAAGCCTTGTTT


In [40]:
# lib = pd.read_csv('OT_Lib_Final.txt', sep = '\t')
lib['prove'] = 0

for i in range(len(lib)):
    if lib.loc[i, 'Seq'][130:150] == lib.loc[i, 'target_gRNA_PAM'][0:20]:
        lib.loc[i, 'prove'] = 1
    else:
        lib.loc[i, 'prove'] = 0

print(lib[lib['prove'] == 0])

                 gRNA_Name              gRNA_seq                   gRNA_PAM  \
9            OR4K15--51501  AAAAAACAACATAAATAGGA  GAAAAAACAACATAAATAGGAAGGg   
12          CIAPIN1--44836  AAAAAACCAAACTTTGAAGT  GAAAAAACCAAACTTTGAAGTGGGg   
14          NDUFA12--43772  AAAAAACCCGTAGATAGCCT  GAAAAAACCCGTAGATAGCCTCGgt   
23              GNAL--7320  AAAAAACTTTGGGACGATGA  GAAAAAACTTTGGGACGATGAAGGg   
28             STRN--18420  AAAAAAGCATTGCCTGACAG  GAAAAAAGCATTGCCTGACAGCGGg   
...                    ...                   ...                        ...   
81314          WDR4--29142  GTTTGTGAGCCGTATCTCCG  GGTTTGTGAGCCGTATCTCCGTGgt   
81320         ADAP1--29891  GTTTGTGCTGACAGAACGAG  GGTTTGTGCTGACAGAACGAGAGgt   
81325         RIPK3--29898  GTTTGTTAACGTAAACCGGA  GGTTTGTTAACGTAAACCGGAAGGg   
81327  hsa-mir-8068--78791  GTTTGTTGTAAGGATCGTTG  GGTTTGTTGTAAGGATCGTTGTGgt   
81328           OS9--29649  GTTTGTTGTTGGCATGATAG  GGTTTGTTGTTGGCATGATAGGGGg   

                        target_seq  \
9      ATGGGA

In [41]:
lib.dtypes

gRNA_Name          object
gRNA_seq           object
gRNA_PAM           object
target_seq         object
Seq                object
target_pos          int64
prefix             object
suffix             object
target_gRNA_PAM    object
count               int64
prove               int64
dtype: object

In [78]:
lib

,gRNA_Name,gRNA_seq,gRNA_PAM,target_seq,Seq,target_pos,prefix,suffix,target_gRNA_PAM,count,prove,ensure
0,hsa-mir-3129--81249,AAAAAAAAAAAGTCGTACTT,GAAAAAAAAAAAGTCGTACTTGGgt,ATGGAAAAAAAAAAAGTCGTACTTGGG,gcgagaaaagccttgtttAAAAAAAAAAAGTCGTACTTgttttaga...,130,gccaccATGG,tgagcaaggg,AAAAAAAAAAAGTCGTACTTGGG,1,1,0
1,hsa-mir-4329--79147,AAAAAAAAACTCCAAAACCC,GAAAAAAAAACTCCAAAACCCTGgt,ATGGAAAAAAAAACTCCAAAACCCTGG,gcgagaaaagccttgtttAAAAAAAAACTCCAAAACCCgttttaga...,130,gccaccATGG,tgagcaaggg,AAAAAAAAACTCCAAAACCCTGG,1,1,0
2,hsa-mir-1303--78858,AAAAAAAAATTGTAGAGTTG,GAAAAAAAAATTGTAGAGTTGAGgt,ATGGAAAAAAAAATTGTAGAGTTGAGG,gcgagaaaagccttgtttAAAAAAAAATTGTAGAGTTGgttttaga...,130,gccaccATGG,tgagcaaggg,AAAAAAAAATTGTAGAGTTGAGG,1,1,0
3,SLC25A24--36887,AAAAAAAATCCGGACAATGG,GAAAAAAAATCCGGACAATGGTGGG,ATGGAAAAAAAATCCGGACAATGGTGG,gcgagaaaagccttgtttAAAAAAAATCCGGACAATGGGttttaga...,130,gccaccATGG,Ggtgagcaag,AAAAAAAATCCGGACAATGGTGG,1,1,0
4,ZNF43--20560,AAAAAAAGACCATAAAAGTG,GAAAAAAAGACCATAAAAGTGTGgt,ATGGAAAAAAAGACCATAAAAGTGTGG,gcgagaaaagccttgtttAAAAAAAGACCATAAAAGTGGttttaga...,130,gccaccATGG,tgagcaaggg,AAAAAAAGACCATAAAAGTGTGG,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
81325,RIPK3--29898,GTTTGTTAACGTAAACCGGA,GGTTTGTTAACGTAAACCGGAAGGg,ATGGGGTTTGTTAACGTAAACCGGAAG,gcgagaaaagccttgtttGTTTGTTAACGTAAACCGGAGttttaga...,131,ccaccATGGG,gtgagcaagg,GTTTGTTAACGTAAACCGGAAGG,1,0,0
81326,BEND6--65735,GTTTGTTGCTCTCCAGAGGG,GGTTTGTTGCTCTCCAGAGGGTGgt,ATGGGTTTGTTGCTCTCCAGAGGGTGG,gcgagaaaagccttgtttGTTTGTTGCTCTCCAGAGGGGttttaga...,130,gccaccATGG,tgagcaaggg,GTTTGTTGCTCTCCAGAGGGTGG,1,1,0
81327,hsa-mir-8068--78791,GTTTGTTGTAAGGATCGTTG,GGTTTGTTGTAAGGATCGTTGTGgt,ATGGTTTGTTGTAAGGATCGTTGTGGT,gcgagaaaagccttgtttGTTTGTTGTAAGGATCGTTGgttttaga...,129,tgccaccATG,tgagcaaggg,GTTTGTTGTAAGGATCGTTGTGG,1,0,0
81328,OS9--29649,GTTTGTTGTTGGCATGATAG,GGTTTGTTGTTGGCATGATAGGGGg,ATGGTTTGTTGTTGGCATGATAGGGGG,gcgagaaaagccttgtttGTTTGTTGTTGGCATGATAGGttttaga...,129,tgccaccATG,gtgagcaagg,GTTTGTTGTTGGCATGATAGGGG,1,0,0


In [95]:
lib['ensure'] = 2

for i in range(len(lib)):
    start_num = lib.loc[i, 'target_pos']
    end_num = start_num + 20
    # print(start_num)
    # print(lib.loc[i, :])
    # print('lib.loc[i, Seq] start_num : end_num :',lib.loc[i, 'Seq'][start_num : end_num])
    # print('lib.loc[i, target_gRNA_PAM][0:20]:',lib.loc[i, 'target_gRNA_PAM'][0:20])
    if lib.loc[i, 'Seq'][start_num : end_num] == lib.loc[i, 'target_gRNA_PAM'][0:20]:
        lib.loc[i,'ensure'] = 1
    else:
        lib.loc[i,'ensure'] = 0

print(lib[lib['ensure'] == 0])

Empty DataFrame
Columns: [gRNA_Name, gRNA_seq, gRNA_PAM, target_seq, Seq, target_pos, prefix, suffix, target_gRNA_PAM, count, prove, ensure]
Index: []


In [105]:
lib_df = lib.copy()
lib_df['50bpSeq'] = 0
lib_df['process'] = 2

for i in range(len(lib_df)):
    start_num = lib_df.loc[i, 'target_pos']
    end_num = start_num + 20
    lib_df.loc[i,'50bpSeq'] = lib_df.loc[i,'Seq'][start_num - 20 : end_num + 10]
    if lib_df.loc[i,'50bpSeq'][20:40] == lib_df.loc[i,'target_gRNA_PAM'][0:20]:
        lib_df.loc[i, 'process'] = 1
    else:
        lib_df.loc[i,'process'] = 0
print(lib_df[lib_df['process'] == 0])

Empty DataFrame
Columns: [gRNA_Name, gRNA_seq, gRNA_PAM, target_seq, Seq, target_pos, prefix, suffix, target_gRNA_PAM, count, prove, ensure, 50bpSeq, process]
Index: []


In [115]:
lib_df['50bpSeq'] = lib_df['50bpSeq'].str.upper()
lib_df = lib_df[['50bpSeq']]
# lib_df
lib_df.to_csv('Lib_with_only_50bp_seq.csv')

In [116]:
lib_df

,50bpSeq
0,GTGCTTTTTTGCCACCATGGAAAAAAAAAAAGTCGTACTTGGGTGA...
1,GTGCTTTTTTGCCACCATGGAAAAAAAAACTCCAAAACCCTGGTGA...
2,GTGCTTTTTTGCCACCATGGAAAAAAAAATTGTAGAGTTGAGGTGA...
3,GTGCTTTTTTGCCACCATGGAAAAAAAATCCGGACAATGGTGGGGT...
4,GTGCTTTTTTGCCACCATGGAAAAAAAGACCATAAAAGTGTGGTGA...
...,...
81325,TGCTTTTTTGCCACCATGGGGTTTGTTAACGTAAACCGGAAGGGTG...
81326,GTGCTTTTTTGCCACCATGGGTTTGTTGCTCTCCAGAGGGTGGTGA...
81327,GGTGCTTTTTTGCCACCATGGTTTGTTGTAAGGATCGTTGTGGTGA...
81328,GGTGCTTTTTTGCCACCATGGTTTGTTGTTGGCATGATAGGGGGTG...


In [81]:
print(lib.loc[12, 'target_pos'])
start_num = lib.loc[12, 'target_pos']
end_num = start_num + 20
print('Seq 131 -> 151:',lib.loc[12, 'Seq'][131 : 151])
print('length of Seq 131 -> 151:',len(lib.loc[12, 'Seq'][131 : 151]))
print('start_num + 1 -> end_num:',lib.loc[12, 'Seq'][start_num + 1 : end_num])
print('length of start_num + 1 -> end_num:',len(lib.loc[12, 'Seq'][start_num + 1 : end_num]))
print('target_seq 0 -> 20:',lib.loc[12, 'target_gRNA_PAM'][0:20])
print('length of target_seq 0 -> 20:',len(lib.loc[12, 'target_gRNA_PAM'][0:20]))
print('===============================')
print(lib.loc[12, 'Seq'][start_num : end_num] == lib.loc[12, 'target_gRNA_PAM'][0:20])


start_num_112 = lib.loc[112, 'target_pos']
end_num_112 = start_num_112 + 20
print(lib.loc[112, 'Seq'][start_num_112 : end_num_112] == lib.loc[112, 'target_gRNA_PAM'][0:20])

131
Seq 131 -> 151: AAAAAACCAAACTTTGAAGT
length of Seq 131 -> 151: 20
start_num + 1 -> end_num: AAAAACCAAACTTTGAAGT
length of start_num + 1 -> end_num: 19
target_seq 0 -> 20: AAAAAACCAAACTTTGAAGT
length of target_seq 0 -> 20: 20
True
True


In [ ]:
# lib = pd.read_csv('OT_Lib_Final.txt', sep = '\t')
lib['makesure'] = 0

for i in range(len(lib)):
    if lib.loc[i, 'Seq'][130:150] == lib.loc[i, 'target_gRNA_PAM'][0:20]:
        lib.loc[i, 'makesure'] = 1
    else:
        lib.loc[i, 'makesure'] = 0

print(lib[lib['prove'] == 0])

In [13]:
print(Lib.iloc[0,0])
print(Lib.iloc[0,0][20:40])
print(len(Lib.iloc[0,0][20:40]))
print(Lib[Lib['Seq'].str[20:40] == 'AAAAAAAAAAAGTCGTACTT'])
print(Lib[Lib['Seq'].str[20:40] == 'AAAAGTAGACATGTCTTCAC'])

GTGCTTTTTTGCCACCATGGAAAAAAAAAAAGTCGTACTTGGGTGAGCAA
AAAAAAAAAAAGTCGTACTT
20
                                                 Seq
0  GTGCTTTTTTGCCACCATGGAAAAAAAAAAAGTCGTACTTGGGTGA...
Empty DataFrame
Columns: [Seq]
Index: []


In [14]:
zcd_ABE_HEK293T = pd.read_csv('ABE_HEK293T_3_10.csv')
zcd_ABE_HEK293T[zcd_ABE_HEK293T['Reference'] == 'AAAAGTAGACATGTCTTCAC']

,Reference,Outcome,Outcome_Count,Total_Count,Outcome_Frequency
897,AAAAGTAGACATGTCTTCAC,AAAAGTAGGCATGTCTTCAC,0,183,0.000000
898,AAAAGTAGACATGTCTTCAC,AAAAGTGGACATGTCTTCAC,70,183,0.382514
899,AAAAGTAGACATGTCTTCAC,AAAAGTGGGCATGTCTTCAC,0,183,0.000000
900,AAAAGTAGACATGTCTTCAC,AAAGGTAGACATGTCTTCAC,3,183,0.016393
901,AAAAGTAGACATGTCTTCAC,AAAGGTAGGCATGTCTTCAC,0,183,0.000000
902,AAAAGTAGACATGTCTTCAC,AAAGGTGGACATGTCTTCAC,3,183,0.016393
903,AAAAGTAGACATGTCTTCAC,AAAGGTGGGCATGTCTTCAC,0,183,0.000000
904,AAAAGTAGACATGTCTTCAC,AAGAGTAGACATGTCTTCAC,0,183,0.000000
905,AAAAGTAGACATGTCTTCAC,AAGAGTAGGCATGTCTTCAC,0,183,0.000000
906,AAAAGTAGACATGTCTTCAC,AAGAGTGGACATGTCTTCAC,0,183,0.000000


In [6]:
def generate_zcd_50bp(Lib, df):
    lib_seqs = Lib['Seq'].tolist()
    lib_seqs_len = len(lib_seqs)
    
    for i in range(len(df)):
        for j in range(lib_seqs_len):
            if df.iloc[i, 0] == lib_seqs[j][20:40]:
                df.loc[i, 'Input'] = lib_seqs[j]
                df.loc[i, 'Output'] = lib_seqs[j][0:20] + df.iloc[i, 1] + lib_seqs[j][40:50]
                break   
                
    return df
                
zcd_ABE_ = zcd_ABE_.copy()
zcd_ABE_ = generate_zcd_50bp(Lib, zcd_ABE_)

In [5]:
zcd_ABE_

,Input,Output,True_Proportion
0,GTGCTTTTTTGCCACCATGGAAAACGGCAAGAAGAACAAGAGGTGA...,GTGCTTTTTTGCCACCATGGAAAACGGCAGGAAGAACAAGAGGTGA...,0.0
1,GTGCTTTTTTGCCACCATGGAAAACGGCAAGAAGAACAAGAGGTGA...,GTGCTTTTTTGCCACCATGGAAAACGGCGAGAAGAACAAGAGGTGA...,0.0
2,GTGCTTTTTTGCCACCATGGAAAACGGCAAGAAGAACAAGAGGTGA...,GTGCTTTTTTGCCACCATGGAAAACGGCGGGAAGAACAAGAGGTGA...,0.0


In [10]:
print(zcd_ABE_['Output'][0])
print(zcd_ABE_['Input'][0])

GTGCTTTTTTGCCACCATGGAAAAACGCAGACATCGTGCAGGGTGAGCAA
GTGCTTTTTTGCCACCATGGAAAAACGCAAACATCGTGCAGGGTGAGCAA


In [11]:
print(Lib.iloc[0,0])
print(len(Lib.iloc[0,0]))

GTGCTTTTTTGCCACCATGGAAAAAAAAAAAGTCGTACTTGGGTGAGCAA
50


In [53]:
print(len(df['gRNA_seq'][0]))
print(len(df['gRNA_PAM'][0]))
print(len(df['target_seq'][0]))
print(len(df['target_gRNA_PAM'][0]))
print(len(df['Seq'][0]))

20
25
27
23
168


In [56]:
print(df['Seq'][0])
print(df['target_gRNA_PAM'][0])q
print(df['Seq'][0][110:160])

gcgagaaaagccttgtttAAAAAAAAAAAGTCGTACTTgttttagagctagaaatagcaagttaaaataaggctagtccgttatcaacttgaaaaagtggcaccgagtcggtgcttttttgccaccATGGAAAAAAAAAAAGTCGTACTTGGgtgagcaagggcgagg
AAAAAAAAAAAGTCGTACTTGGG
gtgcttttttgccaccATGGAAAAAAAAAAAGTCGTACTTGGgtgagcaa


In [10]:
df[df['Seq'].str[130:150] == 'AAAAAGCAGTTTATCCAATG']

NameError: name 'df' is not defined